# Fiktiven Zahlen 
Wir haben nur fiktiven Zahlen benutzt für unsere Simulation, da es bei Stadien bei Konzerten immer auf den Artist ankommt, wie viele Menschen in der Warteschlange sind. Taylor Swift wäre zum Beispiel viel mehr als die KPOP Band Straykids obwohl diese im gleichen Stadion auftreten würden. Daher habenn wir und dazu entschieden nur fiktiven Zahlen für diese Simulation zu benutzen.

# Was soll der Code können? 
- Er soll die Personen zufällig durchlaufen lassen sodass es nich immer die gleiche kaufreihenfolge von 0 bis 4 ist, sondern immer verschieden und zufällig.
- Es sollen auch Personen kaufen die nicht nur für sich Tickets besorgen sondern auch für Freunde/Familie daher kann eine Person auch bis zu 4 Tickets kaufen
- Es gibt auch Personen die in der Warteschlange stehen und sich aufgrund von z.b Ticketpreisen oder begrenzter Platzwahl keine Tickets mehr Kaufen 

# Welche Codes benutzen wir und warum:
numpy für numerische Berechnungen 
pandas für Tabellen und/oder Datenframes
matplotlib für Diagramme

rng=np.random.default_rng(): Soll Zufallgenerator für Zufallszahlen erstellen

def run_single_simulation()...: Soll einzelne Simulation durchführen von den davor angegebenen Daten

summary_positions=...: Soll die Übersichtstabelle für uns erstellen 

cutoff_probs= ... cutoffs=...: Soll für uns die Cutoff Postionen berechnen, das bedeutet die Position an der es unwahrscheinlich ist noch Tickets zu bekommen. 

for k,v in cutoffs.items()...: Soll und die Cutoff-Positionen formatiert ausgeben

plt.plot (positions, p_ticket_left)...: Soll uns ein Diagramm ausgeben das uns die Daten noch anschaulicher macht


# Hier schon der aktuelle Code mit dem wir gerade arbeitn, unten wird beschrieben wie alles funktioniert und wpfür wir das jeweils gemacht haben:

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
#Parameter 
TOTAL_TICKETS=75_000
QUEUE_SIZE=400_000
SIMULATIONS=3_000
TICKET_VALUES=[1,2,3,4]
#Wahrscheinlichkeit für 0 Tickets 
P_ZERO=0.05
#5%-Cutoff
CUTOFF_PROB=0.05
rng=np.random.default_rng()
#Ticketverteilung 1-4
#nicht gleichverteilt
#Gewicht mit 1/sqrt(k)
#ergibt Erwartungswert=1.65
ticket_value=np.array([1,2,3,4])
weights=1/np.sqrt(ticket_value)
weights/= weights.sum()
#Einzelne Simulation
def run_single_simulation():
    tickets_left=TOTAL_TICKETS
    for person in range(QUEUE_SIZE):
        #entscheidet ob Person Tickets kauft
        if rng.random ()<P_ZERO:
            demand=0
        else:
            demand=rng.choice(ticket_value, p=weights)
        tickets_left -=demand
        #Tickets sind aufgebraucht
        if tickets_left<0:
            #letzte Person die noch ein Ticket bekommt
            return person
    return QUEUE_SIZE
#Monte-Carlo-Simulation
sellout_positions=np.array([run_single_simulation()for _ in range(SIMULATIONS)])
#Wahrscheinlichkeit pro Queue-Position
positions=np.arange(0, QUEUE_SIZE +1)
#Wahrscheinlichkeit das es noch Tickets gibt
p_ticket_left=np.array([np.mean(sellout_positions>pos)for pos in positions])
#Tabelle um zu sehen welche Postionen in der Warteschlange welche Wahrscheinlichkeit haben noch Tickets zu bekommen
summary_positions=[20_000,30_000,40_000,50_000,60_000]
table=pd.DataFrame({"Queue-Position":summary_positions, "P(Ticket verfügbar)": [p_ticket_left[pos]for pos in summary_positions], "P(keine Tickets)":[1-p_ticket_left[pos] for pos in summary_positions]})
table["P(Ticket verfügbar)"]=table ["P(Ticket verfügbar)"].round(3)
table["P(Kein Ticket)"]=table["P(Kein Ticket)"].round(3)
print ("\n=Ticket-Chancen nach Queue-Position =\n")
print(table.to_string(index=False))
#Cutoff-Bestimmung 
cutoffs={"50% Chance": np.argmax(p_ticket_left<0.5), "25% Chance": np.argmax(p_ticket_left<0.25), "15% Chance": np.argmax(p_ticket_left<0.15), "10% Chance": np.argmax(p_ticket_left<0.10), "5% Chance": np.argmax(p_ticket_left<0.05), "1% Chance": np.argmax(p_ticket_left<0.01)}
print("\n=Grenze in der es eher Kritisch ist Tickts zu bekommen=\n")
for k, v in cutoffs.items(): 
    print (f"{k:>10}: ca. Position{v:,}")
print("Simulation abgeschlossen")
print(f"Stadion-Tickets:{TOTAL_TICKETS}")
print(f"Simulationen:{SIMULATIONS}")
print(f"Queue-Größe:{QUEUE_SIZE}")
print("\n=Sellout-Postition (Zusammenfassung)=")
print(f"Minimum: {sellout_positions.min():,}")
print(f"5%-Quantil:{int(np.quantile(sellout_positions, 0.05)):,}")
print(f"Median: {int(np.median(sellout_positions)):,}")
print(f"95%-Quantil: {int(np.quantile(sellout_positions, 0.95)):,}")
print(f"Maximum: {sellout_positions.max():,}")


# Parameter:
- TOTAL_TICKETS=75_000 Es gibt insgesamt 75.000 verfügbare Tickets
- QUEUE_SIZE=400_000 Es gibt 400.000 Menschen die in der online Warteschlange warten
- SIMULATIONS=3_000 Wir führen die Simulation insgesamt 3.000 mal durch
- TICKET_VALUES=[1,2,3,4] Pro Person können 1-4 Tickets gekauft werden
- P_ZERO=0.05 Die Wahrscheinlichkeit das jemand kein Ticket kauft liegt bei 5%

# Ticketverteilung: 
Die Ticketanzahl pro Person ist nicht gleichverteilt das muss im Code angegeben werden. Wir wollen hierbei das es wahrscheinlicher ist, dass jemand eher weniger als mehr Tickets kauft, da wir glauben das dies öfter der Fall ist.

In [ ]:
ticket_value=np.array([1,2,3,4])
weights = 1/np.sqrt(ticket_value)
weights /= weights.sum()

-> das bedeutet der Erwartungswert liegt hier bei 1.65 Tickets pro Person

# Einzelne Simulation
Die Funktion soll den Kaufprozess durchlaufen und währendessen die Tickets reduzieren bis keine mehr verfügbar sind.
Wenn die Tickets alle weg sind wird uns die Person angezeigt die kein Ticket mehr bekommen hat (tickets_left<0).
Wenn noch Tickets übrig sein sollten nachdem die komplette Warteschlange durchgelaufen ist wird die Warteschlangen Anzahl zurückgegeben (QUEUE_SIZE)

In [ ]:
def run_single_simulation():
    tickets_left=TOTAL_TICKETS
    for person in range (QUEUE_SIZE):
        # entscheidet ob Person Tickets kauft
        if rng.random()<P_ZERO:
            demand=0
        else: 
            demand=rng.choice(ticket_value, p=weights)
        ticket_left -= demand 
        # Tickets sind aufgebraucht
        if tickets_left<0:
            #Letzte Person, die noch ein Ticket bekommt
            return person
    return QUEUE_SIZE

# Monte-Carlo-Simulation
Wir wollen die Simulation 3000 mal durchführen für ein gutes Ergebnis.

In [ ]:
sellout_positions=np.array([run_single_simulation()for _ in range(SIMULATIONS)])

# Die Wahrscheinlichkeit Tickets zu bekommen
Jetzt wollen wir das unser Code die Wahrscheinlichkeit berechnet noch Tickets zu bekommen.

In [ ]:
# Wahrscheinlichkeit pro Queue-Position
positions=np.array(0, QUEUE_SIZE +1)
# Wahrscheinlichkeit, dass es noch Tickets gibt
p_ticket_left=np.array([np.mean (sellout_positions>pos)for pos in positions])

# Tabelle um eine besseren Überblick bei der Auswertung zu haben 
Wir wollen das die Wahrscheinlichkeiten auf 3 Dezimalstellen gerundet werden (round.3). 
Und die Tabelle soll anzeigen, an welcher Postion in der Warteschlange es wie Wahrscheinlich ist Tickets zu bekommen.

In [ ]:
summary_positions=[20_000,30_000,40_000,50_000,60_000]
table=pd.DataFrame({"Queue-Position":summary_positions, "P(Ticket verfügbar)": [p_ticket_left[pos]for pos in summary_positions], "P(keine Tickets)":[1-p_ticket_left[pos] for pos in summary_positions]})
table["P(Ticket verfügbar)"]=table ["P(Ticket verfügbar)"].round(3)
table["P(Kein Ticket)"]=table["P(Kein Ticket)"].round(3)
print ("\n=Ticket-Chancen nach Queue-Position =\n")
print(table.to_string(index=False))

# Cutoff-Bestimmung
Wir wollen damit die Position finden, an der es unwahrscheinlich ist noch Tickets zu bekommen.

In [ ]:
cutoffs={"50% Chance": np.argmax(p_ticket_left<0.5), "25% Chance": np.argmax(p_ticket_left<0.25), "15% Chance": np.argmax(p_ticket_left<0.15), "10% Chance": np.argmax(p_ticket_left<0.10), "5% Chance": np.argmax(p_ticket_left<0.05), "1% Chance": np.argmax(p_ticket_left<0.01)}
print("\n=Grenze in der es eher Kritisch ist Tickts zu bekommen=\n")

# Jetzt soll alles ausgeführt und anschaulich angezeigt werden

In [ ]:
print("Simulation abgeschlossen")
print(f"Stadion-Tickets:{TOTAL_TICKETS}")
print(f"Simulationen:{SIMULATIONS}")
print(f"Queue-Größe:{QUEUE_SIZE}")
print("\n=Sellout-Postition (Zusammenfassung)=")
print(f"Minimum: {sellout_positions.min():,}")
print(f"5%-Quantil:{int(np.quantile(sellout_positions, 0,05)):,}")
print(f"Median: {int(np.median(sellout_positions)):,}")
print(f"95%-Quantil: {int(np.quantile(sellout_positions, 0,95)):,}")
print(f"Maximum: {sellout_positions.max():,}")


# Wahrscheinlichkeiten besser darstellen in Diagramm

In [ ]:
plt.plot(positions, p_ticket_left)
plt.xlabel("Queue-Position")
plt.ylabel("P(Ticket verfügbar)")
plt.title("Wahrscheinlichkeit, dass noch Tickets verfügbar sind")
plt.grid (True)
plt.show()

# Ergebnisse müssen noch ausgewertet werden

Ergebnisse nach erster Simulation sehen ganz gut aus:
Simulation abgeschlossen
Stadion-Tickets:75.000
Cutoff_Wahrscheinlichkeit: 5.0%
-> Cutoff_Position: ca. Person35.592

Weitere Kennzeichen:
Median:35.764
95%-Quantil:35.937
Minimum:35.456
Maximum:36.093

Ergebnisse nach zweiter und vollständer Simulation:

=Ticket-Chancen nach Queue-Position = 
Queue-Position    P(Ticket vefügbar)    P(Kein Ticket)
20000             1.0                   0.0
30000             1.0                   0.0
40000             0.0                   1.0
50000             0.0                   1.0

=Grenze in der es eher Kritisch ist Tickets zu bekommen=
50% Chance: ca. Position 35,800
25% Chance: ca. Position 35,900
15% Chance: ca. Position 35,900
10% Chance: ca. Position 36,000
5% Chance:  ca. Position 36,000
1% Chance:  ca. Position 36,100
Simulation abgeschlossen
Stadion-Tickets: 75000
Simualtionen: 3000
Queue_Größe:400000

=Sellout-Postition (Zusammenfassung)=
Minimum: 35,408
5%-Quantil: 35,583
Median: 35,761
95%-Quantil: 35,940
Maximum: 36,119
Simulation abgeschlossen
